In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pandas.io import data, wb
import datetime
import os.path
import math


In [23]:
df_train = pd.read_csv('data/spy-ohlc.csv', index_col='Date', parse_dates=True)
df_dividends = pd.read_csv('data/spy-dividends.csv', index_col='Date', parse_dates=True)
df_train = df_train.join(df_dividends)
df_train['dividend'].fillna(0, inplace=True)

In [24]:
#df_train['Close'].pct_change().hist(bins=100)
#plt.show()

In [25]:
pma10 = df_train['Close'].rolling(window=10,center=False).mean() + df_train['dividend'].rolling(window=10, center=False).mean()
pma20 = df_train['Close'].rolling(window=20,center=False).mean()+ df_train['dividend'].rolling(window=20, center=False).mean()
pma50 = df_train['Close'].rolling(window=50,center=False).mean()+ df_train['dividend'].rolling(window=50, center=False).mean()
pma200 = df_train['Close'].rolling(window=200,center=False).mean()+ df_train['dividend'].rolling(window=200, center=False).mean()
vma50 = df_train['Volume'].rolling(window=50, center=False).mean()



In [26]:
#distance from MAs
diffma10  = (df_train['Close']/pma10 -1)*100
diffma20 = (df_train['Close']/pma20 -1)*100
diffma50 = (df_train['Close']/pma50 -1)*100
diffma200 = (df_train['Close']/pma200 -1)*100

#tangential of ma lines
pma10tan = (pma10[1:]/pma10[:-1].values-1)*100
pma20tan = (pma20[1:]/pma20[:-1].values-1)*100
pma50tan = (pma50[1:]/pma50[:-1].values-1)*100
pma200tan = (pma200[1:]/pma200[:-1].values-1)*100

#sequence of higher high/higher low
hhx1 = df_train['High'][1:] + df_train['dividend'][1:] > df_train['High'][:-1] 
hlx1 = df_train['Low'][1:] + df_train['dividend'][1:] > df_train['Low'][:-1] 
hhx2 = hhx1.shift(1)
hhx3 = hhx1.shift(2)
hlx2 = hlx1.shift(1)
hlx3 = hlx1.shift(2)

#rolling average of hh and hl
hh10 = hhx1.rolling(window=10, center=False).mean()
hl10 = hhx1.rolling(window=10, center=False).mean()

nvx1 = df_train['Volume']/vma50


In [27]:
#add features for training data

#distance from MAs
df_train = df_train.assign(diffma10=diffma10)
df_train = df_train.assign(diffma20=diffma20)
df_train = df_train.assign(diffma50=diffma50)
df_train = df_train.assign(diffma200=diffma200)

#tangential of ma lines
df_train = df_train.assign(pma10tan=pma10tan)
df_train = df_train.assign(pma20tan=pma20tan)
df_train = df_train.assign(pma50tan=pma50tan)
df_train = df_train.assign(pma200tan=pma200tan)

df_train = df_train.assign(hhx1=hhx1)
df_train = df_train.assign(hhx2=hhx2)
df_train = df_train.assign(hhx3=hhx3)

df_train = df_train.assign(hlx1=hlx1)
df_train = df_train.assign(hlx2=hlx2)
df_train = df_train.assign(hlx3=hlx3)

df_train = df_train.assign(hh10=hh10)
df_train = df_train.assign(hl10=hl10)

#last day volume
df_train = df_train.assign(nvx1=nvx1)

In [28]:
#df_train.to_csv("data/spy-train.csv")
remove = ['Open','High','Low','Close','Volume','Adj Close','dividend']
df_train = df_train.drop(remove, axis=1)

In [29]:
#df_train.describe()
#df_train.corr()

In [44]:
def predict(X_train, y_train, X_test):  
    from sklearn.cross_validation import train_test_split
    from sklearn import cross_validation
    from sklearn.linear_model import LogisticRegression
    from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.svm import SVC


    #model = LogisticRegression(random_state=1)
    
    #model = RandomForestClassifier(n_estimators=350)
    model = GradientBoostingClassifier(n_estimators=100)
    #best so far
    #model = KNeighborsClassifier(n_neighbors=15)
    
    
    from sklearn import metrics
    
    #y_eval_comp = y_eval_pred[:,1] > 0.55
    #numpy.count_nonzero(y_eval_comp)
    
    #print("recall:", metrics.recall_score(y_eval, y_eval_comp, average='binary'))
    #print("precision:", metrics.precision_score(y_eval, y_eval_comp, average='binary'))
    #print("f1 score:", metrics.f1_score(y_eval, y_eval_comp, average='binary'))
    
    scores = cross_validation.cross_val_score(model, X_train, y_train, cv=5, scoring='precision')
    print("Precision score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))
    
    model.fit(X_train, y_train)
    #print(X_test)
    y_test_pred = model.predict_proba(X_test);
    return y_test_pred



In [45]:
X_train = df_train.values[200:-10, :]
X_test = df_train.values[-10:,:]
print(len(X_train))



5625


In [54]:
#for hh, shift one position back to get future value for hh
#print(hh.shift(-1).tail())
print(len(hhx1), len(hhx1.shift(-1)))
y_train = df_train['hhx1'].shift(-1).values[200:-10].astype(bool)
print(len(y_train))
print("predict hh:(vs actual)")
y_test_pred = predict(X_train, y_train, X_test)
print(df_train['hhx1'].values[-9:].astype(bool))
print(y_test_pred[:, 1]>0.5)
print("prob of higher high ===", y_test_pred[-1,1])

#for hl
print("\npredict hl:(vs actual)")
y_train = df_train['hlx1'].shift(-1).values[200:-10].astype(bool)
y_test_pred = predict(X_train, y_train, X_test)
print(df_train['hlx1'].values[-9:].astype(bool))
print(y_test_pred[:, 1]>0.5)
print("prob of higher low ===", y_test_pred[-1,1])



5834 5834
5625
predict hh:(vs actual)
Precision score: 0.60 (+/- 0.01)
[ True  True  True False False  True  True  True False]
[ True  True  True False False  True  True  True  True False]
prob of higher high === 0.469986023622

predict hl:(vs actual)
Precision score: 0.60 (+/- 0.02)
[ True False False False False  True False  True False]
[ True  True False False False  True  True False False False]
prob of higher low === 0.438397978941


In [33]:
def e_predict(X_train, y_train, X_test):
    from sklearn import cross_validation
    from sklearn.linear_model import LogisticRegression
    from sklearn.naive_bayes import GaussianNB
    from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.ensemble import VotingClassifier

    clf1 = LogisticRegression(random_state=1)
    clf2 = RandomForestClassifier(n_estimators=350)
    clf3 = GaussianNB()
    clf4 = GradientBoostingClassifier(n_estimators=100)
    clf5 = KNeighborsClassifier(n_neighbors=15)
    
    eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3), 
                                        ('gb', clf4), ('knn',clf5)], voting='soft')

    for clf in [clf1, clf2, clf3, clf4, clf5, eclf]: 
        print(type(clf).__name__, "\n")
        scores = cross_validation.cross_val_score(clf, X_train, y_train, cv=5, scoring='precision')
        print("Score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))
        clf.fit(X_train, y_train)
        y_test_pred = clf.predict_proba(X_test);
        return y_test_pred
